In [30]:
import pandas as pd
import os
import numpy as np

In [31]:
# TODO: Write the model name
model = 'Llama-3.1-70B'

In [32]:
# load the data into a single dataframe
df = pd.DataFrame()
for file in os.listdir(f'decision_datasets/{model}/'):
    if file.endswith('.csv'):
        df = pd.concat([df, pd.read_csv(f'decision_datasets/{model}/' + file)], ignore_index=True)

In [33]:
# have a look at the data
df

,id,model,temperature,seed,timestamp,control_options,control_option_order,control_answer,control_extraction,control_decision,treatment_options,treatment_option_order,treatment_answer,treatment_extraction,treatment_decision,status,error_message,individual_score,weight,bias
0,8070,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,42,2024-10-05 17:29:43,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]",After evaluating the given aspects of the adve...,Option 7,6.0,OK,NaN,-1.0,1.0,NegativityBias
1,8071,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,47,2024-10-05 17:29:57,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 6,1.0,OK,NaN,0.4,1.0,NegativityBias
2,8072,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,52,2024-10-05 17:30:11,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 6,5.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 6,5.0,OK,NaN,-0.8,1.0,NegativityBias
3,8073,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,57,2024-10-05 17:30:23,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 7,0.0,OK,NaN,0.5,1.0,NegativityBias
4,8074,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,62,2024-10-05 17:30:38,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the given information, I woul...",Option 4,3.0,OK,NaN,0.0,1.0,NegativityBias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,21295,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,67,2024-10-05 17:46:03,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,OK,NaN,0.0,1.0,SelfServingBias
29996,21296,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,72,2024-10-05 17:46:14,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,OK,NaN,0.0,1.0,SelfServingBias
29997,21297,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,77,2024-10-05 17:46:25,"['To a very large extent', 'To a large extent ...","[0, 1, 2, 3, 4, 5, 6]",I would select Option 2: To a large extent.\n\...,Option 2,1.0,"['To a very large extent', 'To a large extent ...","[0, 1, 2, 3, 4, 5, 6]",I would choose Option 2: To a large extent.\n\...,Option 2,1.0,OK,NaN,0.0,1.0,SelfServingBias
29998,21298,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,82,2024-10-05 17:46:34,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]","As a supply chain manager, I believe that my p...",Option 6,1.0,OK,NaN,0.0,1.0,SelfServingBias


In [34]:
# check for missing decisions: should be 1000 for each bias
df.groupby('bias')['id'].count()

bias
Anchoring                      1000
Anthropomorphism               1000
AvailabilityHeuristic          1000
BandwagonEffect                1000
ConfirmationBias               1000
Conservatism                   1000
DispositionEffect              1000
EndowmentEffect                1000
EscalationOfCommitment         1000
FramingEffect                  1000
FundamentalAttributionError    1000
HaloEffect                     1000
HindsightBias                  1000
HyperbolicDiscounting          1000
IllusionOfControl              1000
InGroupBias                    1000
InformationBias                1000
LossAversion                   1000
MentalAccounting               1000
NegativityBias                 1000
NotInventedHere                1000
OptimismBias                   1000
PlanningFallacy                1000
Reactance                      1000
RiskCompensation               1000
SelfServingBias                1000
SocialDesirabilityBias         1000
StatusQuoBias          

In [36]:
# get the number of completed tests
len(df[df['status'] == 'OK'])

29887

In [38]:
# get the number of completed tests
#df[df['status'] == 'OK'].groupby('bias')['id'].count().sort_values(ascending=False)
# in percentage
(df[df['status'] == 'OK'].groupby('bias')['id'].count().sort_values(ascending=False) / df.groupby('bias')['id'].count() * 100).sort_values(ascending=False)

bias
Anchoring                      100.0
Anthropomorphism               100.0
StatusQuoBias                  100.0
SocialDesirabilityBias         100.0
SelfServingBias                100.0
RiskCompensation               100.0
Reactance                      100.0
PlanningFallacy                100.0
OptimismBias                   100.0
NotInventedHere                100.0
NegativityBias                 100.0
MentalAccounting               100.0
LossAversion                   100.0
InformationBias                100.0
InGroupBias                    100.0
IllusionOfControl              100.0
HyperbolicDiscounting          100.0
HindsightBias                  100.0
HaloEffect                     100.0
FundamentalAttributionError    100.0
FramingEffect                  100.0
EscalationOfCommitment         100.0
EndowmentEffect                100.0
DispositionEffect              100.0
Conservatism                   100.0
ConfirmationBias               100.0
AvailabilityHeuristic          10

In [39]:
# evaluate the biasedness of the model
df_success = df[df['status'] == 'OK']

df_success.groupby(df_success['bias']).apply(lambda x: np.average(x['individual_score'], weights=x['weight'])).sort_values(ascending=False)

/tmp/ipykernel_543926/3520039201.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_success.groupby(df_success['bias']).apply(lambda x: np.average(x['individual_score'], weights=x['weight'])).sort_values(ascending=False)


bias
SurvivorshipBias               0.740700
InformationBias                0.692983
Anchoring                      0.635980
ConfirmationBias               0.613650
MentalAccounting               0.563333
FramingEffect                  0.456750
HyperbolicDiscounting          0.435700
LossAversion                   0.416018
HaloEffect                     0.388433
PlanningFallacy                0.297196
AvailabilityHeuristic          0.278048
Conservatism                   0.234100
IllusionOfControl              0.228747
HindsightBias                  0.223796
SelfServingBias                0.215133
InGroupBias                    0.208867
Stereotyping                   0.195112
SocialDesirabilityBias         0.188533
FundamentalAttributionError    0.164933
RiskCompensation               0.146807
EscalationOfCommitment         0.143341
NotInventedHere                0.127017
NegativityBias                 0.126317
BandwagonEffect                0.088131
OptimismBias                   0.01

In [40]:
df

,id,model,temperature,seed,timestamp,control_options,control_option_order,control_answer,control_extraction,control_decision,treatment_options,treatment_option_order,treatment_answer,treatment_extraction,treatment_decision,status,error_message,individual_score,weight,bias
0,8070,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,42,2024-10-05 17:29:43,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]",After evaluating the given aspects of the adve...,Option 7,6.0,OK,NaN,-1.0,1.0,NegativityBias
1,8071,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,47,2024-10-05 17:29:57,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 6,1.0,OK,NaN,0.4,1.0,NegativityBias
2,8072,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,52,2024-10-05 17:30:11,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 6,5.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 6,5.0,OK,NaN,-0.8,1.0,NegativityBias
3,8073,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,57,2024-10-05 17:30:23,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[6, 5, 4, 3, 2, 1, 0]","After evaluating the situation, I would choose...",Option 7,0.0,OK,NaN,0.5,1.0,NegativityBias
4,8074,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,62,2024-10-05 17:30:38,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the situation, I would choose...",Option 4,3.0,"['Very likely negative aspects', 'Likely negat...","[0, 1, 2, 3, 4, 5, 6]","After evaluating the given information, I woul...",Option 4,3.0,OK,NaN,0.0,1.0,NegativityBias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,21295,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,67,2024-10-05 17:46:03,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,OK,NaN,0.0,1.0,SelfServingBias
29996,21296,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,72,2024-10-05 17:46:14,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,OK,NaN,0.0,1.0,SelfServingBias
29997,21297,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,77,2024-10-05 17:46:25,"['To a very large extent', 'To a large extent ...","[0, 1, 2, 3, 4, 5, 6]",I would select Option 2: To a large extent.\n\...,Option 2,1.0,"['To a very large extent', 'To a large extent ...","[0, 1, 2, 3, 4, 5, 6]",I would choose Option 2: To a large extent.\n\...,Option 2,1.0,OK,NaN,0.0,1.0,SelfServingBias
29998,21298,meta-llama/Meta-Llama-3.1-70B-Instruct,0.0,82,2024-10-05 17:46:34,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]",I would select Option 6: To a large extent.\n\...,Option 6,1.0,"['To a very large extent', 'To a large extent ...","[6, 5, 4, 3, 2, 1, 0]","As a supply chain manager, I believe that my p...",Option 6,1.0,OK,NaN,0.0,1.0,SelfServingBias


In [29]:
df.to_csv(f'decision_datasets/{model}.csv', index=False)